# Imports

In [1]:
import numpy as np
from ete3 import Tree

C:\Users\karna\anaconda3\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\karna\anaconda3\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


# Variables

In [2]:
step_matrix = np.array([[0, 3, 4, 9],
                       [3, 0, 2, 4],
                       [4, 2, 0, 4],
                       [9, 4, 4, 0]])

n_species = step_matrix.shape[0]

# Ordre : A C G T
dic1 = {
          'Probopass':np.array([0,np.inf,np.inf,np.inf]),
          'Aggron':np.array([np.inf,np.inf,np.inf,0]),
          'Bastiodon':np.array([np.inf,np.inf,np.inf,0]),
          'Regirock':np.array([np.inf,np.inf,0,np.inf]),
          'Registeel':np.array([np.inf,np.inf,0,np.inf]),
          'Regice':np.array([np.inf,np.inf,0,np.inf]),
          'Klingklang':np.array([np.inf,np.inf,0,np.inf]),
          'Metagross':np.array([np.inf,0,np.inf,np.inf]),
          'Genesect':np.array([0,np.inf,np.inf,np.inf]),
          'Porygon=Z':np.array([np.inf,0,np.inf,np.inf]),
          'Magnezone':np.array([np.inf,0,np.inf,np.inf]),
          'Forretress':np.array([np.inf,np.inf,np.inf,0]),
          'Electrode':np.array([0,np.inf,np.inf,np.inf]),
          'Ferrothorn':np.array([np.inf,np.inf,0,np.inf]),
       }

N1 = "(((( Electrode , Magnezone) ,Porygon=Z) , (((( Aggron , Bastiodon ) , Forretress ) , Ferrothorn ) , ((((( Regirock , Regice ) , Registeel ) , Metagross ) , Klingklang ) , Genesect ))) , Probopass );"
N2 = "((((( Regirock , Regice ) , Registeel ) , (( Metagross , Klingklang ) , Genesect )) , ((( Aggron , Bastiodon ) ,( Forretress , Ferrothorn )) , Probopass )) ,( Porygon=Z,( Magnezone , Electrode )));"

# Trees

<div>
<img src="tree1.png" width="500px">
<img src="tree2.png" width="500px">
</div>

# Sankoff algorithm

In [3]:
tree1 = Tree(N1)
print(N1)
print(tree1)

(((( Electrode , Magnezone) ,Porygon=Z) , (((( Aggron , Bastiodon ) , Forretress ) , Ferrothorn ) , ((((( Regirock , Regice ) , Registeel ) , Metagross ) , Klingklang ) , Genesect ))) , Probopass );

            /-Electrode
         /-|
      /-|   \-Magnezone
     |  |
     |   \-Porygon=Z
     |
     |            /-Aggron
     |         /-|
     |      /-|   \-Bastiodon
   /-|     |  |
  |  |   /-|   \-Forretress
  |  |  |  |
  |  |  |   \-Ferrothorn
  |  |  |
  |  |  |               /-Regirock
  |  |  |            /-|
  |   \-|         /-|   \-Regice
--|     |        |  |
  |     |      /-|   \-Registeel
  |     |     |  |
  |     |   /-|   \-Metagross
  |     |  |  |
  |      \-|   \-Klingklang
  |        |
  |         \-Genesect
  |
   \-Probopass


In [217]:
def sankoff(tree):
    tree = tree.replace(" ","")
    tree = tree.replace(";","")
    split_tree = tree.split(",")
    species = []
    for elt in split_tree:
        elt = elt.replace("(","") 
        elt = elt.replace(")","")
        species.append(elt)
    index = 0
    n_parenthesis = 0
    for char in tree:
        
        if char == "(":
#             print(n_parenthesis)
            n_parenthesis += 1
        elif char == ")":  
            print(n_parenthesis)
            print(tree)
            print(species)
            print(species[index])
            print(species[index+1])
            print("============")
            n_parenthesis -= 1
            tmp_ancester = add_ancester(species[index], species[index+1])
            species.remove(species[index])
            species.remove(species[index])
            species.insert(index, tmp_ancester)

def add_ancester(elt1, elt2):
    new_ancester = elt1+"-"+elt2    
    new_tab = compute_new_values(elt1,elt2)
    dic1[new_ancester] = new_tab
    return new_ancester
    
def compute_new_values(elt1,elt2):
    new_tab = np.zeros((n_species))
    val1 = dic1[elt1]
    val2 = dic1[elt2]
    for i in range(n_species):
        val_letter = step_matrix[i]
        new_tab[i] = np.min(val_letter+val1) + np.min(val_letter+val2)
    return new_tab

In [218]:
sankoff(N1)

4
((((Electrode,Magnezone),Porygon=Z),((((Aggron,Bastiodon),Forretress),Ferrothorn),(((((Regirock,Regice),Registeel),Metagross),Klingklang),Genesect))),Probopass)
['Electrode', 'Magnezone', 'Porygon=Z', 'Aggron', 'Bastiodon', 'Forretress', 'Ferrothorn', 'Regirock', 'Regice', 'Registeel', 'Metagross', 'Klingklang', 'Genesect', 'Probopass']
Electrode
Magnezone
3
((((Electrode,Magnezone),Porygon=Z),((((Aggron,Bastiodon),Forretress),Ferrothorn),(((((Regirock,Regice),Registeel),Metagross),Klingklang),Genesect))),Probopass)
['Electrode-Magnezone', 'Porygon=Z', 'Aggron', 'Bastiodon', 'Forretress', 'Ferrothorn', 'Regirock', 'Regice', 'Registeel', 'Metagross', 'Klingklang', 'Genesect', 'Probopass']
Electrode-Magnezone
Porygon=Z
6
((((Electrode,Magnezone),Porygon=Z),((((Aggron,Bastiodon),Forretress),Ferrothorn),(((((Regirock,Regice),Registeel),Metagross),Klingklang),Genesect))),Probopass)
['Electrode-Magnezone-Porygon=Z', 'Aggron', 'Bastiodon', 'Forretress', 'Ferrothorn', 'Regirock', 'Regice', '